## Quadrotor Maze with Visualization

In [1]:
using TrajectoryOptimization
using Plots, LinearAlgebra, MeshCat, GeometryTypes, CoordinateTransformations, FileIO, MeshIO

┌ Info: Recompiling stale cache file /home/taylor/.julia/compiled/v1.1/TrajectoryOptimization/UVgeA.ji for TrajectoryOptimization [c79d492b-0548-5874-b488-5a62c1d9d0ca]
└ @ Base loading.jl:1184


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/c443b012d0ad3ce8e6cc9de306cd3766b1d18a79-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/c443b012d0ad3ce8e6cc9de306cd3766b1d18a79-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Import the quadrotor model

In [2]:
model = Dynamics.quadrotor_model
n = model.n # number of states
m = model.m; # number of controls

Define numerical type

In [3]:
T = Float64;

Define initial and goals states

In [4]:
q0 = [1.;0.;0.;0.] # unit quaternion

x0 = zeros(T,n)
x0[1:3] = [0.; 0.; 10.]
x0[4:7] = q0

xf = zero(x0)
xf[1:3] = [0.;60.; 10.]
xf[4:7] = q0;

Define a cost function, e.g., quadratic

In [5]:
Q = (1.0e-4)*Diagonal(I,n)
R = (1.0e-3)*Diagonal(I,m)
Qf = 1000.0*Diagonal(I,n)
_cost = LQRCost(Q, R, Qf, xf);

Define constraints

In [6]:
r_quad = 3.
r_cylinder = 2.
cylinders = []
zh = 3
l1 = 5
l2 = 4
l3 = 5
l4 = 10

for i = range(-25,stop=-10,length=l1)
    push!(cylinders,(i, 10,r_cylinder))
end

for i = range(10,stop=25,length=l1)
    push!(cylinders,(i, 10, r_cylinder))
end

for i = range(-7.5,stop=7.5,length=l3)
    push!(cylinders,(i, 30, r_cylinder))
end

for i = range(-25,stop=-10,length=l1)
    push!(cylinders,(i, 50, r_cylinder))
end

for i = range(10,stop=25,length=l1)
    push!(cylinders,(i, 50, r_cylinder))
end

for i = range(10+2*r_cylinder,stop=50-2*r_cylinder,length=l4)
    push!(cylinders,(-25, i, r_cylinder))
end

for i = range(10+2*r_cylinder,stop=50-2*r_cylinder,length=l4)
    push!(cylinders,(25, i, r_cylinder))
end

n_cylinders = length(cylinders)

function cI_maze(c,x,u)
    for i = 1:n_cylinders
        c[i] = circle_constraint(x,cylinders[i][1],cylinders[i][2],cylinders[i][3]+r_quad)
    end
end

maze = Constraint{Inequality}(cI_maze,n,m,n_cylinders,:maze)

u_min = 0.
u_max = 20.
x_max = Inf*ones(model.n)
x_min = -Inf*ones(model.n)

x_max[1:3] = [25.0; Inf; 20]
x_min[1:3] = [-25.0; -Inf; 0.]
bnd = BoundConstraint(n,m,u_min=u_min,u_max=u_max,x_min=x_min,x_max=x_max,trim=true)
# bnd = BoundConstraint(n,m,x_min=x_min,x_max=x_max,trim=true)

goal = goal_constraint(xf)
con = [bnd,maze,goal]; # constraint set

Solver options

In [7]:
verbose=false
opts_ilqr = iLQRSolverOptions{T}(verbose=true,iterations=300,live_plotting=:off)

opts_al = AugmentedLagrangianSolverOptions{T}(verbose=true,opts_uncon=opts_ilqr,
    iterations=20,cost_tolerance=1.0e-6,cost_tolerance_intermediate=1.0e-5,constraint_tolerance=1.0e-3,penalty_scaling=10.,penalty_initial=1.)

opts_altro = ALTROSolverOptions{T}(verbose=true,resolve_feasible_problem=true,opts_al=opts_al,R_inf=0.001);

Define a problem

In [8]:
N = 101 # number of knot points
tf = 5.0
dt = tf/(N-1) # total time

U = [0.5*9.81/4.0*ones(m) for k = 1:N-1] # initial hovering control trajectory
obj = Objective(_cost,N) # objective with same stagewise costs

con_set = Constraints(con,N) # constraint trajectory

prob = Problem(model,obj, constraints=con_set, x0=x0, integration=:rk4, N=N, dt=dt)
initial_controls!(prob,U); # initialize problem with controls

State trajectory guess

In [9]:
X_guess = zeros(n,7)
X_guess[:,1] = x0
X_guess[:,7] = xf
X_guess[1:3,2:6] .= [0 -12.5 -20 -12.5 0 ;15 20 30 40 45 ;10 10 10 10 10]

X_guess[4:7,:] .= q0
X0 = TrajectoryOptimization.interp_rows(N,tf,X_guess);
copyto!(prob.X,X0);

Solve problem

In [10]:
solve!(prob, opts_altro)

┌ Info: Infeasible Solve
└ @ TrajectoryOptimization /home/taylor/.julia/dev/TrajectoryOptimization/src/altro.jl:64
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     0.2691162456  15.851988   0.010504  0.25      0e+00     0.166502  4.296544 0          
    2     0.2679929302  0.15894424  0.007067  0.003906  0e+00     0.001123  4.215328 0          
    3     0.2490184844  4.75189571  0.003993  0.125     0e+00     0.018974  4.30264  0          
    4     0.2486403579  0.07225218  0.005233  0.001953  0e+00     0.000378  4.935833 0          
    5     0.1567623567  8.08159641  0.011369  0.25      0e+00     0.091878  4.911801 0          
    6     0.1562258404  0.07036623  0.007625  0.003906  0e+00     0.000537  3.329569 0          
    7    

    19    0.3173739218  7.5195151   0.009653  0.125     0e+00     0.072585  1.448145 0          
    20    0.3173704322  0.01212895  0.000288  0.000244  0e+00     3.49e-06  1.111348 0          
iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
5     0       0.02354228  0.31737  
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     1.8954225478  0.72034236  0.010181  0.001953  0e+00     0.007334  5.036663 0          
    2     1.8881409199  0.71748808  0.010149  0.001953  0e+00     0.007282  5.031666 0          
    3     1.8809110823  0.71465409  0.010117  0.001953  0e+00     0.00723   5.025676 0          
    4     1.8

    13    3.509573179   1.35884376  0.007704  0.001953  0e+00     0.010469  5.218036 0          
    14    3.4992895883  1.35477339  0.007591  0.001953  0e+00     0.010284  5.311987 0          
    15    3.4891910247  1.35077481  0.007476  0.001953  0e+00     0.010099  5.412752 0          
    16    3.4792749529  1.34684774  0.007362  0.001953  0e+00     0.009916  5.523658 0          
    17    3.4695385225  1.34299093  0.00725   0.001953  0e+00     0.009736  5.650256 0          
    18    3.4599793429  1.33920307  0.007138  0.001953  0e+00     0.009559  5.797844 0          
    19    3.4505648107  1.33548394  0.00705   0.001953  0e+00     0.009415  5.983567 0          
    20    3.4448781409  0.66624069  0.008535  0.000977  0e+00     0.005687  6.250839 0          
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____-------------------------------------------------------------------------

    76    0.414757185   0.38611223  0.015776  0.007813  0e+00     0.006091  0.996217 0          
    77    0.4087563908  0.3801063   0.015787  0.007813  0e+00     0.006001  0.987605 0          
    78    0.4028452622  0.37419362  0.015797  0.007813  0e+00     0.005911  0.978919 0          
    79    0.390306232   0.73385627  0.017086  0.015625  0e+00     0.012539  0.970072 0          
    80    0.3846631834  0.35685735  0.015813  0.007813  0e+00     0.005643  0.972539 0          
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    81    0.3735089019  0.69985544  0.015938  0.015625  0e+00     0.011154  0.963455 0          
    82    0.3656185143  0.67815284  0.011635  0.015625  0e+00     0.00789   0.95758  0          
    83    0.3563060451  0.6

    49    1.8178812423  0.12033227  0.002249  0.000977  0e+00     0.000271  0.967584 0          
    50    1.8176430892  0.1207109   0.001973  0.000977  0e+00     0.000238  0.966877 0          
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    51    1.8174380676  0.12109508  0.001693  0.000977  0e+00     0.000205  0.966763 0          
    52    1.8172661539  0.12148492  0.001415  0.000977  0e+00     0.000172  0.96595  0          
    53    1.8171391776  0.1218804   0.001042  0.000977  0e+00     0.000127  0.965591 0          
    54    1.8170392512  0.12228383  0.000817  0.000977  0e+00     9.99e-05  0.965322 0          
    55    1.8169623587  0.12269175  0.000627  0.000977  0e+00     7.69e-05  0.964994 0          
    56    1.8169257707  0.1

    11    14.513947363  1.56279425  0.01299   0.001953  0e+00     0.0203    0.996408 0          
    12    14.496452106  1.56985947  0.011144  0.001953  0e+00     0.017495  0.998257 0          
    13    14.48407459   1.57797849  0.007844  0.001953  0e+00     0.012378  0.998723 0          
    14    14.472196603  1.58791442  0.00748   0.001953  0e+00     0.011878  1.0031   0          
    15    14.4623205    1.59814209  0.00618   0.001953  0e+00     0.009876  1.001521 0          
    16    14.457682613  1.60916008  0.002882  0.001953  0e+00     0.004638  1.004372 0          
    17    14.451419968  1.62204125  0.003861  0.001953  0e+00     0.006263  1.004361 0          
    18    14.4499756    1.63440581  0.000884  0.001953  0e+00     0.001444  1.005722 0          
    19    14.435086828  0.82467382  0.018054  0.000977  0e+00     0.014889  1.00641  0          
    20    14.420723028  0.82563202  0.017397  0.000977  0e+00     0.014364  1.008042 0          
    iter cost          expecte

    75    13.823813494  0.90943913  0.008319  0.000977  0e+00     0.007566  0.986326 0          
    76    13.8156972    0.91143718  0.008905  0.000977  0e+00     0.008116  0.985591 0          
    77    13.80863217   0.91332277  0.007736  0.000977  0e+00     0.007065  0.983014 0          
    78    13.801798703  0.91540485  0.007465  0.000977  0e+00     0.006833  0.984146 0          
    79    13.794430418  0.91752507  0.008031  0.000977  0e+00     0.007368  0.983925 0          
    80    13.787419916  0.91953522  0.007624  0.000977  0e+00     0.007011  0.98348  0          
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    81    13.781252005  0.92160904  0.006693  0.000977  0e+00     0.006168  0.982386 0          
    82    13.774181334  0.9

    139   6.7375454152  4.27963455  0.024988  0.007813  0e+00     0.106938  0.44736  0          
    140   6.6322224687  4.2144216   0.024991  0.007813  0e+00     0.105323  0.444083 0          
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    141   6.5284718005  4.15018726  0.024999  0.007813  0e+00     0.103751  0.436904 0          
    142   6.4262971851  4.08687309  0.025001  0.007813  0e+00     0.102175  0.433799 0          
    143   6.3256832663  4.02448689  0.025     0.007813  0e+00     0.100614  0.430093 0          
    144   6.226553341   3.96303109  0.025014  0.007813  0e+00     0.09913   0.424847 0          
    145   6.1289303249  3.90242559  0.025016  0.007813  0e+00     0.097623  0.421859 0          
    146   6.0327950692  3.8

Resolving feasible


    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     0.0835824635  0.00004548  0.088489  0.001953  0e+00     4.02e-06  0.167332 0          
iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
1     0       0.06859697  0.083582 
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     0.0783133878  0.04607302  0.126794  0.5       0e+00     0.005842  0.075513 0          
    2     

Visualization

In [11]:
vis = Visualizer()
# open(vis)
IJuliaCell(vis)

WebIO.Scope("scope-29bfe614-176b-4b5a-b9c0-cf3a9cc1e44c", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :iframe), Any[], Dict{Symbol,Any}(:attributes=>Dict{String,Any}("height"=>"100%","scrolling"=>"no","src"=>"javascript:void(0)","frameborder"=>0),:id=>"ifr",:style=>Dict("width"=>"100%","flexGrow"=>"1")), 0)], Dict{Symbol,Any}(:style=>Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden")), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}(), Set(String[]), nothing, Any[], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Ensure that the iframe's baseURI matches the baseURI of the\n    // outer document. This is necessary to resolve\n    // https://github.com/JuliaGizmos/WebIO.jl/issues/167\n    var base = doc.createElement(\"base\");\n    base.setAttribute(\"href\", document.baseURI);\n    doc.head.appendChild(base);\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"/assetserver/c443b012d0ad3ce8e6cc9de306cd3766b1d18a79-assets/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;/assetserver/39091d6b9eb892b592bedf9cea6c288f61f7d3f3-main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-811fcc65-140e-41b1-ba7b-07809c0b1f5c&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-811fcc65-140e-41b1-ba7b-07809c0b1f5c&quot;,&quot;id&quot;:&quot;ob_02&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); window.document.body.style.margin=&quot;0&quot;; return window.meshcat_viewer=this.viewer})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_01&quot;,&quot;value&quot;:[]},&quot;meshcat-controls&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_03&quot;,&quot;value&quot;:[]},&quot;meshcat-request&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_02&quot;,&quot;value&quot;:&quot;&quot;}}},&quot;children&quot;:[{&quot;props&quot;:{&quot;className&quot;:&quot;meshcat-viewer&quot;,&quot;style&quot;:{&quot;left&quot;:0,&quot;right&quot;:0,&quot;overflow&quot;:&quot;hidden&quot;,&quot;position&quot;:&quot;absolute&quot;,&quot;height&quot;:&quot;100vh&quot;,&quot;margin&quot;:0,&quot;width&quot;:&quot;100vw&quot;}},&quot;nodeType&quot;:&quot;DOM&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;namespace&quot;:&quot;html&quot;,&quot;tag&quot;:&quot;div&quot;},&quot;children&quot;:[]}]})</unsafe-script>\" + \"</body></html>\";\n        setTimeout(function () { resizeIframe() }, 0)\n    }\n\n    doc.body.appendChild(webio)\n}"), JSString("(function (){retur

Colors

In [12]:
green_ = MeshPhongMaterial(color=RGBA(0, 1, 0, 1.0))
green_transparent = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.1))
red_ = MeshPhongMaterial(color=RGBA(1, 0, 0, 1.0))
red_transparent = MeshPhongMaterial(color=RGBA(1, 0, 0, 0.1))
blue_ = MeshPhongMaterial(color=RGBA(0, 0, 1, 1.0))
blue_transparent = MeshPhongMaterial(color=RGBA(0, 0, 1, 0.1))
blue_semi = MeshPhongMaterial(color=RGBA(0, 0, 1, 0.5))
yellow_ = MeshPhongMaterial(color=RGBA(1, 1, 0, 1.0))
yellow_transparent = MeshPhongMaterial(color=RGBA(1, 1, 0, 0.75))

orange_ = MeshPhongMaterial(color=RGBA(233/255, 164/255, 16/255, 1.0))
orange_transparent = MeshPhongMaterial(color=RGBA(233/255, 164/255, 16/255, 0.1))
black_ = MeshPhongMaterial(color=RGBA(0, 0, 0, 1.0))
black_transparent = MeshPhongMaterial(color=RGBA(0, 0, 0, 0.1))
black_semi = MeshPhongMaterial(color=RGBA(0, 0, 0, 0.5));

In [13]:
function plot_cylinder(c1,c2,radius,mat,name="")
    geom = Cylinder(Point3f0(c1),Point3f0(c2),convert(Float32,radius))
    setobject!(vis["cyl"][name],geom,red_)
end

function addcylinders!(vis,cylinders,height=1.5)
    for (i,cyl) in enumerate(cylinders)
        plot_cylinder([cyl[1],cyl[2],0],[cyl[1],cyl[2],height],cyl[3],blue_,"cyl_$i")
        # plot_cylinder([cyl[1],cyl[2],0],[0,0,height],cyl[3],blue_,"cyl_$i")
    end
end

addcylinders! (generic function with 2 methods)

In [14]:
traj_folder = joinpath(dirname(pathof(TrajectoryOptimization)),"..")
urdf_folder = joinpath(traj_folder, "dynamics","urdf")
obj = joinpath(urdf_folder, "quadrotor_base.obj")

robot_obj = FileIO.load(obj);

In [15]:
sphere_small = HyperSphere(Point3f0(0), convert(Float32,0.1*r_quad)) # trajectory points
sphere_medium = HyperSphere(Point3f0(0), convert(Float32,0.25*r_quad));

In [16]:
obstacles = vis["obs"]
traj = vis["traj"]
robot = vis["robot"]
setobject!(vis["robot"]["quad"],robot_obj,black_);

In [17]:
settransform!(vis["/Cameras/default"], compose(Translation(0., 72., 60.),LinearMap(RotX(pi/7.5)*RotZ(pi/2))))

MeshCat Visualizer with path /Cameras/default

In [18]:
addcylinders!(vis,cylinders,16.0)

In [19]:
traj = vis["traj"]

for i = 1:N
    setobject!(vis["traj"]["t$i"],sphere_small,blue_)
    settransform!(vis["traj"]["t$i"], Translation(prob.X[i][1], prob.X[i][2], prob.X[i][3]))
end

In [20]:
anim = MeshCat.Animation(20)
for i = 1:N
    MeshCat.atframe(anim,vis,i) do frame
        settransform!(frame["robot"], compose(Translation(prob.X[i][1:3]...),LinearMap(Quat(prob.X[i][4:7]...))))
    end
end
MeshCat.setanimation!(vis,anim)